In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict
import time

#Load file in a data frame
df = pd.read_csv('training_data.csv', nrows=500000)


In [2]:
df.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...


In [2]:
#################TIME TEST################
df['product_action'] = df['product_action'].fillna('view')


In [4]:
df = df.groupby('session_id_hash')[['server_timestamp_epoch_ms', 'product_action']].agg(list).reset_index()
df.head()

,session_id_hash,server_timestamp_epoch_ms,product_action
0,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[1552054733254, 1552054754795, 1552054795153, ...","[view, view, view, detail, view, view, detail,..."
1,00010d84aca1294479304044207fd268f63228844779c6...,"[1548008872581, 1548009161452, 1548009170805, ...","[view, view, view, detail, view, view, view, v..."
2,000123fbafc4db26c25b29b9f26c72bb4efdca3a28d1c6...,"[1548464608066, 1548464646689, 1548464646689]","[view, detail, view]"
3,0001ae7726ce0a9caeab048f586f2078e7cfa44fd53837...,"[1548343013689, 1548343069779, 1548343069779, ...","[view, detail, view, view, view, view]"
4,00074ee1d03117b4179bb64e90de5a4ae2c7c84e0a99e4...,"[1552584260337, 1552584260337]","[detail, view]"


In [32]:
#Test: Convert epoch time to regular time
x = df['server_timestamp_epoch_ms'][0]
y = []
for times in x:
    y.append(time.localtime(times/1000)) #Divide by 1000 to convert from miliseconds to regular time


In [15]:
#Create a new column with the "regular" time format
df['session_time'] = df.server_timestamp_epoch_ms.map(lambda x: time.localtime(x/1000)) #this needs to iterate over the 
                                                                                        #the values in the list


SyntaxError: invalid syntax (<ipython-input-15-2f310cd6c14e>, line 1)

In [5]:
#Doesn't work. Length of y is higher than df index
y = []
for times in df['server_timestamp_epoch_ms']:
    for x in times:
        y.append(time.localtime(x/1000))

In [ ]:
##########################TIMES TEST END############################

In [18]:
# Sessionise:
# derive sessions from action by action dataset
# merge all the actions by session_id

df['product_action'] = df['product_action'].fillna('view')
df = df.groupby('session_id_hash')['product_action'].agg(list).reset_index()
df.head()

,session_id_hash,product_action
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi..."
1,000009f36a40de1d557afc083dbb3fc03eef2473337bad...,"[view, view]"
2,00000e812c3076d18245710a31b348d3f23314b7d0dc90...,[view]
3,00001355930ff05e66ab30bccff221c33eba90e1517397...,"[view, detail]"
4,0000162d1dad0beb867c191ab2c8c7c06086cc57d9ebe2...,"[view, view, view, view, detail, view, detail,..."


In [19]:
## select sessions with at least one add-to-cart
df = df[df.product_action.map(lambda x: "add" in x)]

In [20]:
##  labelization
df['purchase'] = np.where(df.product_action.map(set(['purchase']).issubset), 1, 0)
df.head()

,session_id_hash,product_action,purchase
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi...",0
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, view, view, detail, view, view, detail,...",0
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, detail, view, view, view, v...",0
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, detail, add, view, view, detail, add, v...",0
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[view, view, view, view, detail, add, remove, ...",1


In [5]:
#Cutting purchase sessions to last event before the firts purchase
sequence_action = []
for session in data['product_action']:
    if 'purchase' in set(session):
        for e in range(len(session)):
            if session[e] == 'purchase':
                sequence_action.append(session[:e])
                break
    else:
        sequence_action.append(session)
data['session'] = sequence_action
data = data[['session_id_hash', 'session', 'purchase']]


In [22]:
#2nd method for cutting sessions
df['session'] = df['product_action'].map(lambda x: x[0:(x.index('purchase'))] if 'purchase' in x else x)
df = df[['session_id_hash', 'session', 'purchase']]

In [25]:
#Removing outliers
df['len'] = df.session.map(len)
df.drop(df[df.len < 5].index, inplace=True)
df.drop(df[df.len > 155].index, inplace=True)
df.head()

,session_id_hash,session,purchase,len
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi...",0,18
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, view, view, detail, view, view, detail,...",0,139
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, detail, view, view, view, v...",0,41
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, detail, add, view, view, detail, add, v...",0,16
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[view, view, view, view, detail, add, remove, ...",1,29


In [26]:
#Symbolization
from collections import Counter

sessions = df['session'].to_list()
labels = df['purchase'].to_list()

counts = Counter([item for session in sessions for item in session])
symbol2idx = {symbol: idx for idx, symbol in enumerate(sorted(counts, key=counts.get, reverse=True), 1)}
print(counts)
print(symbol2idx)

symbolised_sessions = []
for idx, session in enumerate(sessions):
    symbolised_session = [symbol2idx[s] for s in session]
    symbolised_sessions.append(symbolised_session)
    
df['Symbolised'] = symbolised_sessions
df.head()

Counter({'view': 3014880, 'detail': 1010529, 'add': 246971, 'remove': 217562})
{'view': 1, 'detail': 2, 'add': 3, 'remove': 4}


,session_id_hash,session,purchase,len,Symbolised
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi...",0,18,"[1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, ..."
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, view, view, detail, view, view, detail,...",0,139,"[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, ..."
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, detail, view, view, view, v...",0,41,"[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ..."
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, detail, add, view, view, detail, add, v...",0,16,"[1, 2, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[view, view, view, view, detail, add, remove, ...",1,29,"[1, 1, 1, 1, 2, 3, 4, 1, 2, 1, 1, 1, 4, 4, 4, ..."


In [11]:
#Function that splits the data by filtering the length of the Symbolised session sequence

def split_length(length):
    filtered = data.session.apply(lambda x: len(x) == length)
    filtered_data = data[filtered]
    symbolised = filtered_data['session']
    filtered_labels = filtered_data['purchase']
    
    train_ids = np.random.choice(
    len(symbolised), int(np.floor(len(symbolised)*0.7)), replace=False
    )
    training_sessions, training_labels, eval_sessions, eval_labels = [], [], [], []
    for idx, (session, label) in enumerate(zip(symbolised, filtered_labels)):
        if idx in train_ids:
            training_sessions.append(session)
            training_labels.append(label)
        else:
            eval_sessions.append(session)
            eval_labels.append(label)
    return training_sessions, training_labels, eval_sessions, eval_labels

training_sessions, training_labels, eval_sessions, eval_labels,  = split_length(10) 

In [151]:
#Oracle
from collections import defaultdict

def oracle():
    d = defaultdict(lambda: defaultdict(int))
    for session, label in zip(training_sessions, training_labels):
        d[str(tuple(session))][label] += 1
        
    oracle_dict = {}
    for session in d.keys():
        n_buy, n_ws = 0, 0
        for label in d[session].keys():
            if label:
                n_buy += d[session][label]
            else:
                n_ws += d[session][label]
        oracle_dict[session] = n_buy / (n_buy + n_ws)
    return oracle_dict    


In [154]:
#Calculate F1 score for Oracle
orax = oracle()
pred_label = []
string_sessions = []

#Appending evaluation sessions to a list in the same format (strings) 
#as the sessions in the oracle dictionary
for x in eval_sessions:
    string_sessions.append(str(tuple(x)))

#Map and append the values in the oracle to a list based on the 
#evaluation sessions
for label in string_sessions:
    if label in orax.keys():
        pred_label.append(int(orax[label]))
    else:
        pred_label.append(0)

f1_score(eval_labels, pred_label)

0.07947019867549668

In [10]:
#Split data

def convert_session(x):
    return [int(i) for i in x.strip("[]").split(",")]

data.session = data.session.map(convert_session)

In [30]:
df.head()

,session_id_hash,session,purchase,len,Symbolised
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,"[view, detail, add, view, view, view, view, vi...",0,18,"[1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, ..."
37,0000913afa22ba9c31efb992bcf6388b0bbfe28056bef3...,"[view, view, view, detail, view, view, detail,...",0,139,"[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, ..."
64,00010d84aca1294479304044207fd268f63228844779c6...,"[view, view, view, detail, view, view, view, v...",0,41,"[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ..."
84,0001368d732951035a7ef7ef42b345a5c50b7d66966749...,"[view, detail, add, view, view, detail, add, v...",0,16,"[1, 2, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
119,0001c180fb742f96ff388ba8f67a568e6fa66aed30d0d2...,"[view, view, view, view, detail, add, remove, ...",1,29,"[1, 1, 1, 1, 2, 3, 4, 1, 2, 1, 1, 1, 4, 4, 4, ..."


In [43]:
#split data
data = defaultdict(lambda: defaultdict())

for i in [5, 10, 15]:
    
    ## Subset from the original df
    temp = df[df.len == i]
    temp["session"] = temp.session.map(lambda x: x[0:i])
    temp = temp[["Symbolised", "purchase"]]
    #temp_x = temp[['session', 'len']]
    
    ## Split train and test
    X_train, X_test, y_train, y_test = train_test_split(temp['Symbolised'], temp["purchase"], test_size=0.2, random_state=123)
    data[f"{i}_clicks"]["X_train"] = X_train.to_numpy()
    data[f"{i}_clicks"]["X_test"] = X_test.to_numpy()
    data[f"{i}_clicks"]["y_train"] = y_train.to_numpy()
    data[f"{i}_clicks"]["y_test"] = y_test.to_numpy()

<ipython-input-43-e5d0209fe05f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["session"] = temp.session.map(lambda x: x[0:i])
<ipython-input-43-e5d0209fe05f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["session"] = temp.session.map(lambda x: x[0:i])
<ipython-input-43-e5d0209fe05f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [44]:
#Markov chain
from collections import defaultdict

def get_ngram(session, i, n):
        
        """
        Takes in a list, an index and the ngram size n, and returns the history and current 
        token of the appropriate size: the current token is the one at the provided index, while the 
        history consists of the n-1 previous tokens. If the ngram size is 1, only the current token is returned.
        """
        
        if n == 1:
            return session[i]
        else:
            ngram = session[i-(n-1):i+1]
            history = tuple(ngram[:-1])
            target = ngram[-1]
            return (history, target)
        
                    
def update_counts(sessions, n):
        
    """
    Processes the input sessions given an ngram sizes and stores transition counts. 
    A dictionary of dictionary is created, where the first level key indicates the history, the second level 
    key indicates the current word, and the value indicates the history-current word co-occurrence count.
    If the ngram size is 1, then a simple dictionary mapping unigram to frequency count is created
    """
        
    counts = defaultdict(dict)
    for session in sessions:
        session = ['#']*(n-1) + session + ['+']
        for idx in range(n-1, len(session)):
            ngram = get_ngram(session, idx, n)
            if n == 1:
                counts[ngram] += 1
            else:
                # it's faster to try to do something and catch an exception than to use an if statement to 
                # check whether a condition is met beforehand. The if is checked everytime, the exception 
                # is only catched the first time, after that everything runs smoothly
                try:
                    counts[ngram[0]][ngram[1]] += 1
                except KeyError:
                    counts[ngram[0]][ngram[1]] = 1
        
    # first loop through the sentences in the corpus, than loop through each word in a sentence
    vocab = {event for session in sessions for event in session}
        
    return counts, vocab


In [45]:
data['5_clicks']['X_test']

array([list([1, 2, 1, 3, 1]), list([1, 1, 3, 1, 1]),
       list([2, 1, 3, 1, 1]), ..., list([1, 2, 3, 3, 1]),
       list([1, 2, 3, 1, 1]), list([1, 1, 2, 3, 1])], dtype=object)

In [46]:
train_buy, train_ws = [], []
for session, label in zip(data['10_clicks']['X_train'], data['10_clicks']['y_train']):
    if label == 1:
        train_buy.append(session)
    else:
        train_ws.append(session)

In [47]:
buy_counts, buy_vocab = update_counts(train_buy, 10)
ws_counts, ws_vocab = update_counts(train_ws, 10)

In [48]:
def get_prior(labels):
    
    """
    Given a vector of binary labels (0 and 1), computes the probability of the positive class.
    """
    
    return np.mean(labels)

def get_posterior(prior, likelihood_one, likelihood_zero):
    
    """
    Given the prior of the positive class and the likelihood of an input under the language models trained
    on input from the positive and negative classes, returns the posterior probability of the positive class.
    """
    
    if 0 < likelihood_zero <= 1 and 0 < likelihood_one <= 1:
        return (prior * likelihood_one) / ((prior * likelihood_one) + ((1 - prior) * likelihood_zero))
    elif likelihood_zero <= 0 and likelihood_one <= 0:
        prior = np.log(prior) if 0 < prior <= 1 else prior
        normalize = np.logaddexp(prior + likelihood_one, 1 - prior + likelihood_zero)
        return np.exp(prior + likelihood_one - normalize)
    else:
        raise ValueError("One or both likelihood terms are higher than 1!")
    

def get_unigram_probability(ngram, counts, vocab_size, k=1):
        
    """
    
    """
        
    tot = sum(list(counts.values())) + (vocab_size*k)
        
    try:
        ngram_count = counts[ngram] + k
    except KeyError:
        ngram_count = k
        
    return ngram_count/tot

def get_ngram_probability(history, target, counts, vocab_size, k=1):
        
    """
    
    """
        
    try:
        ngram_tot = np.sum(list(counts[history].values())) + (vocab_size*k)
        try:
            transition_count = counts[history][target] + k
        except KeyError:
            transition_count = k
    except KeyError:
        transition_count = k
        ngram_tot = vocab_size*k
            
    return transition_count/ngram_tot 

def get_sequence_prob(session, counts, n, vocab_size, k=1):
        
    """
        
    """
        
    probs = []
    for idx in range(n-1, len(session)):
        ngram = get_ngram(session, idx, n)
        if n == 1:
            probs.append(get_unigram_probability(ngram, counts, vocab_size))
        else:
            probs.append(get_ngram_probability(ngram[0], ngram[1], counts, vocab_size))

        return np.prod(probs)

In [50]:
markov_predictions = []
prior_buy = get_prior(data['10_clicks']['y_train'])
for session in data['10_clicks']['X_test']:
    likelihood_buy = get_sequence_prob(session, buy_counts, 10, len(buy_vocab))
    likelihood_ws = get_sequence_prob(session, ws_counts, 10, len(ws_vocab))

    markov_predictions.append(np.round(get_posterior(prior_buy, likelihood_buy, likelihood_ws)))

In [52]:
from sklearn import metrics
acc = metrics.accuracy_score(data['10_clicks']['y_test'], markov_predictions)
f1 = metrics.f1_score(data['10_clicks']['y_test'], markov_predictions)
print(acc, f1)

0.7956331877729258 0.016806722689075633
